# Data Mining Versuch Recommender Systeme

* Autor: Prof. Dr. Johannes Maucher
* Datum: 30.09.2015

[Übersicht Ipython Notebooks im Data Mining Praktikum](Data Mining Praktikum.ipynb)


# Einführung
## Lernziele:
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

* __Ähnlichkeit:__ Verfahren zur Bestimmung der Ähnlichkeit zwischen Personen (Kunden) und Elementen (Produkten)
* __Empfehlungssysteme__ Collaborative Filtering 
* __Collaborative Filtering:__ Nutzerbezogener Ansatz und elementbasierter Ansatz

Sämtliche Verfahren und Algorithmen werden in Python implementiert.

## Theorie zur Vorbereitung
### Recommender Systeme
Recommender Systeme werden im E-Commerce eingesetzt um Werbung in Form von kundenspezifischen Empfehlungen zu verteilen. Weitläufig bekannt sind die Amazon-Empfehlungen, die entweder per e-mail geschickt oder nach dem Log-In in der Web-Page angezeigt werden. Diese Empfehlungen werden in Abhängigkeit von den bisher vom jeweiligen Kunden gekauften bzw. bewerteten Produkten erstellt. In diesem Versuch werden die derzeit wohl am weitest verbreiteteten Verfahren für die Erzeugung kundenspezifischer Empfehlungen vorgestellt, darunter das elementweise Collaborative Filtering, welches z.B. auch von Amazon eingesetzt wird.     

Direkt-Marketing Methoden wie die kundenspezifische Erzeugung und Bereitstellung von Werbung erfordern detaillierte Kunden- und Warenkorbanalysen. Kunden mit ähnlichem Kaufverhalten werden in Kundengruppen zusammengefasst. Die Warenkorbanalyse untersucht u.a. welche Waren bevorzugt im Verbund von der gleichen Person gekauft werden. Damit kann ein Händler Werbung in Form von Empfehlungen individuell und gezielt an seine Kunden richten, abhängig davon welcher Kundengruppe er angehört und welche Produkte bevorzugt von dieser Kundengruppe nachgefragt werden. 

Im ersten Teil der Übung werden fiktive Daten in einer überschaubaren Menge verwendet. Es handelt sich hier um Filmbewertungen. Anhand dieses Beispiels sollen die notwendigen Methoden und Abläufe implementiert und getestet werden. Diese werden im zweiten Teil der Übung auf echte Daten angewandt. Hierzu werden über eine Python-API Daten vom Internet-Meta-Radio _last.fm_ integriert. Auf der Basis dieser Daten sollen dann Musikempfehlungen für last.fm User berechnet werden. 

Recommender Systeme lassen sich mit

* Clustering Verfahren
* Suchalgorithmen
* Collaborativen Filtering 
 
realisieren. Am häufigsten wird hierbei das Collaborative Filtering eingesetzt. Für das Collaborative Filtering wird jeder der $M$ User durch einen $N$-dimensionalen Vektor beschrieben, wobei $N$ die Anzahl der Produkte im Angebot des Händlers ist. Jedes Element im Vektor gehört zu einem speziellen Produkt. Das Element hat den Wert 1, wenn der User dieses Produkt bereits gekauft hat, sonst 0 (andere Wertbelegungen sind möglich, z.B. wenn Produktbewertungen vorliegen). Alle $M$ Zeilenvektoren können zur _User/Item_ Matrix zusammengefasst werden (siehe Abbildung).

![Abbildung User Item Matrix](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining//Bilder/UserItemMatrix.png "User Item Matrix")

Das traditionelle __userbasierte Collaborative Filtering (UCF)__, benutzt die Ähnlichkeit zwischen Benutzern: Um für User $U_i$ eine Empfehlung zu erzeugen wird zunächst der diesem User ähnlichste Kunde (oder eine Menge vom ähnlichsten Kunden) ermittelt. Dann werden $U_i$ die Produkte (Items) empfohlen, welche der ähnlichste Kunde gekauft hat, $U_i$ selbst jedoch noch nicht. 

Dieser Ansatz skaliert schlecht im Fall sehr großer _User/Item_-Matrizen. Ausserdem ist er für User, welche erst wenige Produkte gekauft haben unzuverlässig. Besser eignet sich in diesen Fällen das __itembasierte Collaborative Filtering (ICF)__. Es wird u.a. von Amazon.com eingesetzt. Diese Variante benutzt die Ähnlichkeit zwischen Produkten (Items). Dabei sind Produkte umso ähnlicher je mehr Kunden diese Produkte gemeinsam gekauft haben. Für die Produkte welche ein Referenzuser $U_i$ bereits gekauft hat, werden die ähnlichsten Produkte ermittelt. Diese ähnlichsten Produkte werden $U_i$ empfohlen, wenn er sie nicht schon selbst gekauft hat.

Im folgenden Abschnitt werden einige gebräuchliche Metriken für die Berechnung der Ähnlichkeit zwischen Benutzern oder Artikeln vorgestellt. Für Collaboratives Filtering wird sehr häufig das Cosinus - Ähnlichkeitsmaß eingesetzt.


### Gebräuchliche Ähnlichkeitsmaße

Die __euklidische Distanz__ $d_E(\underline{a},\underline{b})$ zwischen zwei n-dimensionalen Vektoren $\underline{a}=(a_1,\ldots,a_n)$ und $\underline{b}=(b_1,\ldots,b_n)$ berechnet sich zu
	$$
	d_E(\underline{a},\underline{b})=\sqrt{\sum_{i=1}^n (a_i-b_i)^2}
	$$
Zwei Vektoren können als umso ähnlicher erachtet werden, je kleiner deren euklidische Distanz ist. 
Ein auf der euklidischen Metrik basierendes Ähnlichkeitsmaß zwischen zwei Vektoren $\underline{a}$ und $\underline{b}$ kann durch 
$$
s_E(\underline{a},\underline{b})=\frac{1}{1+d_E(\underline{a},\underline{b})}
$$
angegeben werden.


__Pearson Korrelation__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Pearson-Korrelationskoeffizient $\rho_{\underline{a},\underline{b}}$ ausgedrückt werden. Er berechnet sich zu
$$
\rho_{\underline{a},\underline{b}}= \frac{1}{N}\cdot \sum\limits_{i=1}^{N}\frac{(a_i-\overline{a})}{\sigma_a} \frac{(b_i-\overline{b})}{\sigma_b}
$$
Dabei bezeichnet $N$ die Länge der Vektoren, $\overline{a}$ den Mittelwert und $\sigma_a$ die Standardabweichung des Vektors $\underline{a}$. 

Der Pearson-Korrelationskoeffizient misst die lineare Abhängigkeit zwischen zwei Vektoren. Der maximale Wert von $+1$ wird erreicht, wenn die durch die beiden Vektoren definierten N Punkte im 2-dimensionalen Raum auf einer ansteigenden Geraden liegen. Der Minimalwert von $-1$ wird erreicht, wenn die Punkte auf einer abfallenden Geraden liegen. Der Betrag des Koeffizienten ist umso kleiner, je stärker die Punkte von einer fiktiven Geraden (kann durch lineare Regression berechnet werden) abweichen. Der Koeffizient ist $0$ wenn keine lineare Abhängigkeit zwischen den Vektoren besteht.


__Cosinus Ähnlichkeitsmaß__
Die Ähnlichkeit zwischen zwei Vektoren kann auch durch den Cosinus $\cos(\underline{a},\underline{b})$ ausgedrückt werden. Er berechnet sich zu
$$
\cos(\underline{a},\underline{b})= \frac{\underline{a} \cdot \underline{b}}{\left\|\underline{a}\right\|\cdot \left\|\underline{b}\right\|}
$$
wobei im Zähler das Skalarprodukt der beiden Vektoren steht und mit $\left\|\underline{x}\right\|$ der Betrag des Vektors $\underline{x}$ bezeichnet wird.

Falls die Vektoren $\underline{a}$ und $\underline{b}$ mittelwertfrei sind, ist der Cosinus-Ähnlichkeitswert gleich dem Pearson-Korrelationswert. In der Dokument- und Textanalyse wird vornehmlich das Cosinus-Ähnlichkeitsmaß verwendet. 


__Russel Rao Ähnlichkeitsmaß__
Die Russel Rao-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert 1 haben und der Länge $n$ der Vektoren. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Russel-Rao-Ähnlichkeit $s_{RR}(\underline{a},\underline{b})=2/6=0.333$.

__Jaccard Ähnlichkeitsmaß__
Die Jaccard-Ähnlichkeit zwischen zwei binären Vektoren $\underline{a}$ und $\underline{b}$ mißt das Verhältnis zwischen der Anzahl $\alpha$ der Stellen in denen beide Vektoren den Wert $1$ haben und der Anzahl der Stellen in denen mindestens einer der beiden Vektoren ungleich $0$ ist. Z.B. ist für die Vektoren $\underline{a}=(1,0,1,0,0,1)$ und $\underline{b}=(0,1,1,1,0,1)$ die Jaccard-Ähnlichkeit $s_{J}(\underline{a},\underline{b})=2/5=0.4$. Die Jaccard Metrik wird in diesem Versuch für die Bestimmung der Ähnlichkeit von _last.fm_-Usern eingesetzt.


## Vor dem Versuch zu klärende Fragen
Eine Untermenge der im Folgenden aufgeführten Fragen wird zu Beginn des Versuchs im Rahmen eines Gruppenkolloqs abgefragt. Auf jede Frage sollte von mindestens einem Gruppenmitglied eine Antwort geliefert werden und jedes Gruppenmitglied muss mindestens eine der gestellten Fragen beantworten können.

**Aufgaben:**

* Beschreiben Sie das Prinzip des userbasierten Collaborativen Filtering (UCF).

Hierbei werden Verhaltensmuster von Benutzergruppen ausgewertet, um auf die Interessen Einzelner zu schliessen.

Beim userbasierten Collaborativen Filtering wird für einen Kunden A, dem eine Empfehlung gemacht werden soll, 
der ähnlichste Kunde B, bzw. eine Menge von ähnlichsten Kunden ermittelt. 
Dem Kunden A werden dann die Produkte empfohlen, die er noch nicht gekauft hat, aber Kunde B bereits gekauft hat.

* Welche Nachteile hat das UCF?

Umso größer die Kunden-Item-Matrix ist, desto schlechter skaliert der UCF-Ansatz. 
Auch für Kunden, die erst wenig Produkte gekauft haben, ist der Ansatz eher schlecht geeignet. 
Denn umso weniger Produkte ein Kunde gekauft hat, desto schwerer ist es, einen ähnlichen Kunden zu finden.

* Worin besteht der Unterschied zwischen UCF und itembasierten Collaborativen Filtering (ICF)?

Das UCF vegleicht den Datenbestand zweier User, wohingegen das ICF die Ähnlichkeit zwischen Produkten nutzt. 
Beim ICF muss man nur einzelne Produktklassen betrachten, daher skaliert das ICF weitaus besser mit großen Datenmengen.

Beispiel:   Eine Festplatte der Firma Seagate ist ein ähnliches Produkt wie eine Festplatte der Firma Samsung. 
            A und viele andere User haben die Seagate Festplatte gekauft. Ein Großteil der anderen Benutzer 
            haben aber auch die Samsung Festplatte gekauft. A besitzt die Samsung Festplatte jedoch nicht, 
            daher wird ihm diese Festplatte Empfohlen.

* Gegeben seien die Vektoren 

    \begin{eqnarray*}
    \underline{a} & = & [1,2,3,4,5,6] \\
    \underline{b} & = & [3,3,5,6,7,8] \\
    \end{eqnarray*}
    
    Schreiben Sie eine Python Funktion, die den Mittelwert derartiger Vektoren berechnet. Schreiben Sie eine weitere Funktion, die die Varianz berechnet

#### calculateMean - Funktion, die den Mittelwert berechnet
Alle Elemente des übergebenen Vektors werden summiert und die Summe anschließend durch die Anzahl der Elemente des Vektors geteilt.

In [1]:
def calculateMean(vector):
    sum = 0
    for element in vector:
        sum += element
    result = float(sum) / float(len(vector))
    return result

#### variance - Funktion, die die Varianz berechnet
Der Mittelwert des Vektors wird mit der bereits implementierten Funktion *calculateMean* ermittelt. Anschließend wird die Summe aller quadrierten Differenzen zwischen jedem Element und dem Mittelwert gebildet. Diese Summe wird durch die Anzahl Elemente des Vektors geteilt.

In [2]:
def variance(vector):
    mean = calculateMean(vector)
    sum = 0
    for element in vector:
        sum += (element-mean)**2
    result = float(sum) / float(len(vector))
    return result

#### Berechnung von Mittelwert und Varianz der Vektoren a und b mithilfe der selbst geschriebenen Funktionen

In [3]:
a = [1, 2, 3, 4, 5, 6]
b = [3, 3, 5, 6, 7, 8]

print "Mean a: ", calculateMean(a)
print "Mean b: ", calculateMean(b)
print "Variance a: ", variance(a)
print "Variance b: ", variance(b)

Mean a:  3.5
Mean b:  5.33333333333
Variance a:  2.91666666667
Variance b:  3.55555555556


#### Berechnung von Mittelwert und Varianz der Vektoren a und b mithilfe von Numpy

In [4]:
#Python Code 02 with numpy
    #Konzept
        
        #Mittelwert a:
        #   (1 + 2 + 3 + 4 + 5 + 6) / 6
        #   = 21 / 6
        #   = 3.5
        #Varianz a:
        #   ( (1-3.5)²+(2-3.5)²+(3-3.5)²+(4-3.5)²+(5-3.5)²+(6-3.5)² ) / 6
        #   17.5 / 6
        #   2.92
        
#Mittelwert:
import numpy as np
a = np.array(a)
b = np.array(b)
aMean = np.mean(a)
print 'Mittelwert a:',aMean
#3,5
bMean = np.mean(b)
print 'Mittelwert b',bMean
#5,3333

#Varianz:
aVar = np.var(a)
print 'Varianz a',aVar
#2,92
bVar = np.var(b)
print 'Varianz b',bVar
#3,5

Mittelwert a: 3.5
Mittelwert b 5.333333333333333
Varianz a 2.9166666666666665
Varianz b 3.5555555555555554


* Wie groß ist die

    - Euklidische Ähnlichkeit
    - Pearson Ähnlichkeit
    - Cosinus Ähnlichkeit
    
    zwischen den Vektoren $\underline{a}$ und $\underline{b}$? 

#### Euklidische Ähnlichkeit
$d(euclidean)=\sqrt{(1-3)^2+(2-3)^2+(3-5)^2+(4-6)^2+(5-7)^2+(6-8)^2}=\sqrt{21}$<br>
$s(euclidean)=\frac{1}{1+d(euclidean)}=\frac{1}{1+\sqrt{21}}=\frac{1}{1+4.58}=0.18$<br>

---

#### Pearsonähnlichkeit
- Mittelwert von a = 3.5
- Mittelwert von b = 5.333
- Varianz von a = 2.9167
- Varianz von b = 3.5556
- Standardabweichung von a = $\sqrt{2.9167}$
- Standardabweichung von a = $\sqrt{3.556}$

$s(pearson)=\frac{1}{6}*(\frac{(1-3.5)*(3-5.3)}{\sqrt{2.9167}*\sqrt{3.556}}+
              \frac{(2-3.5)*(3-5.3)}{\sqrt{2.9167}*\sqrt{3.556}}+
              \frac{(3-3.5)*(5-5.3)}{\sqrt{2.9167}*\sqrt{3.556}}+
              \frac{(4-3.5)*(6-5.3)}{\sqrt{2.9167}*\sqrt{3.556}}+
              \frac{(5-3.5)*(7-5.3)}{\sqrt{2.9167}*\sqrt{3.556}}+
              \frac{(6-3.5)*(8-5.3)}{\sqrt{2.9167}*\sqrt{3.556}}) = 0.983$

---

#### Cosinusähnlicheit
$s(cosinus)=\frac{\vec{a}*\vec{b}}{||\vec{a}||*||\vec{b}||}=\frac{\{1, 2, 3, 4, 5, 6\}*\{3, 3, 5, 6, 7, 8\}}{\sqrt{1^2+2^2+3^2+4^2+5^2+6^2}*\sqrt{3^2+3^2+5^2+6^2+7^2+8^2}}=\frac{131}{9.599*13.856}=0.991$

* In welchen Fällen sind Cosinus- und Pearsonähnlichkeit der euklidischen Ähnlichkeit vorzuziehen?

Cosinus und Peasonähnlichkeit eignet sich besonders für skalierende Werte, 
da es nicht die Distanz zu 2 Werten berechnet sondern den Abstand von einem Idealwert und darüberhinaus Informationen 
über die Richtung angibt. Cosinus und Pearsonähnlichkeit eignet sich daher für Collaborative Fitting.

In [5]:
from IPython.display import Latex
from IPython.display import Image
import pylast

# Versuchsdurchführung
## Teil 1: Fiktive Filmbewertung
### Daten
Folgende Tabelle enthält die Filmbewertungen von 7 Personen.
from IPython.display import Latex
In diesem Versuch sollen Kenntnisse in folgenden Themen vermittelt werden:

![Abbildung Bewertung Fiktive Kunden](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining/Bilder/recommenderFilmRecommendations.PNG)

Die Tabelle ist als Python dictionary _critics_ implementiert. Die Keys des Python-Dictionary definieren die Namen von Personen (Zeilen in der Matrix), die Filme bewertet haben. Die Values sind selbst wieder Dictionarys, welche als Keys die Filmnamen (Spalten in der Matrix) und als Values die jeweilige Filmbewertung (Matrixelment) enthalten.

In [6]:
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 
 'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 
 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 
 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
 'The Night Listener': 4.5, 'Superman Returns': 4.0, 
 'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 
 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}
}

### Ähnlichkeiten berechnen

Für die Bestimmung der Ähnlichkeit zwischen Personen und Produkten werden in diesem Versuch ein auf der euklidischen Distanz basierendes Ähnlichkeitsmaß und die Pearson-Korrelation verwendet. Beide Ähnlichkeitsmaße sind in den unten definierten Funktionen implementiert. Alle drei hier implementierten Funktionen zur Berechnung der Ähnlichkeit erhalten als Übergabeparameter das oben definierte Dictionary, das die Filmbewertungen enthält und die Namen der zwei Personen, die verglichen werden sollen. 

Zu beachten ist, dass in beiden Funktionen für die Berechnung der Ähnlichkeit zwischen zwei Personen nur die Produkte berücksichtigt werden, welche von beiden Personen schon bewertet wurden. Es handelt sich hier also um modifizierte Ähnlichkeitsfunktionen. 

__Aufgabe:__
Fragen Sie von diesem Dictionary _Toby's_ Bewertung des Films _Snakes on a Plane_ ab und geben Sie diesen Wert aus: 

In [7]:
print '-'*51,"Toby's critic",'-'*51,'\n', 'Snakes on a Plane: ', critics['Toby']['Snakes on a Plane']
print '-'*117

--------------------------------------------------- Toby's critic --------------------------------------------------- 
Snakes on a Plane:  4.5
---------------------------------------------------------------------------------------------------------------------


In [8]:
import numpy as np
import scipy.spatial.distance as sci


def sim_euclid(prefs,person1,person2,normed=True):
  ''' Returns a euclidean-distance-based similarity score for 
  person1 and person2. In the distance calculation the sum is computed 
  only over those items, which are nonzero for both instances, i.e. only
  films which are ranked by both persons are regarded.
  If the parameter normed is True, then the euclidean distance is divided by
  the number of non-zero elements integrated in the distance calculation. Thus
  the effect of larger distances in the case of an increasing number of commonly ranked
  items is avoided.
  '''
  # Get the list of shared_items
  si={}
  for item in prefs[person1]: 
    if item in prefs[person2]: si[item]=1
  # len(si) counts the number of common ratings
  # if they have no ratings in common, return 0
  if len(si)==0: return 0

  # Add up the squares of all the differences
  sum_of_squares=np.sqrt(sum([pow(prefs[person1][item]-prefs[person2][item],2) 
                     for item in prefs[person1] if item in prefs[person2]]))
  if normed:
     sum_of_squares= 1.0/len(si)*sum_of_squares
  return 1/(1+sum_of_squares)


def sim_pearson(prefs,p1,p2):
  '''
  Returns the Pearson correlation coefficient for p1 and p2
  '''
    
  # Get the list of commonly rated items
  si={}
  for item in prefs[p1]: 
    if item in prefs[p2]: si[item]=1

  # if they are no ratings in common, return 0
  if len(si)==0: return 0

  # Sum calculations
  n=len(si)
  
  # Calculate means of person 1 and 2
  mp1=np.mean([prefs[p1][it] for it in si])
  mp2=np.mean([prefs[p2][it] for it in si])
  
  # Calculate standard deviation of person 1 and 2
  sp1=np.std([prefs[p1][it] for it in si])
  sp2=np.std([prefs[p2][it] for it in si])
  
  # If all elements in one sample are identical, the standard deviation is 0. 
  # In this case there is no linear correlation between the samples
  if sp1==0 or sp2==0:
      return 0
  r=1/(n*sp1*sp2)*sum([(prefs[p1][it]-mp1)*(prefs[p2][it]-mp2) for it in si])
  return r


def sim_RusselRao(prefs,person1,person2,normed=True):
  ''' Returns RusselRao similaritiy between 2 users. The RusselRao similarity just counts the number
  of common non-zero components of the two vectors and divides this number by N, where N is the length
  of the vectors. If normed=False, the division by N is omitted.
  '''
  # Get the list of shared_items
  si={}
  commons=0
  for item in prefs[person1]: 
    if prefs[person1][item]==1 and prefs[person2][item]==1:   
        commons+=1
  #print commons
  if not normed:
      return commons
  else:
      return commons*1.0/len(prefs[person1]) 

**Aufgabe:**
1. Geben Sie die euklidische Ähnlichkeit und die Pearson Ähnlichkeit zwischen den Personen _Toby_ und _Lisa Rose_ aus.
2. Diskutieren Sie die unterschiedlichen Ähnlichkeitswert
3. Welches Ähnlichkeitsmaß erscheint Ihnen für diese Anwendung am besten geeignet?

#### Aufgabe 1

In [9]:
print '-'*47,'euclidean similarity','-'*47,'\n', 'Toby & Lisa Rose: ', sim_euclid(critics,'Toby','Lisa Rose')
print '-'*48,'pearson similarity','-'*48,'\n', 'Toby & Lisa Rose: ', sim_pearson(critics,'Toby','Lisa Rose')
print '-'*116

----------------------------------------------- euclidean similarity ----------------------------------------------- 
Toby & Lisa Rose:  0.615911621788925
------------------------------------------------ pearson similarity ------------------------------------------------ 
Toby & Lisa Rose:  0.9912407071619302
--------------------------------------------------------------------------------------------------------------------


#### Aufgabe 2

Die Ähnlichkeiten unterscheiden sich so deutlich, da die euklidische Distanz den persönlichen Offset eines Users nicht beachtet. Besonders bei Bewertungen tritt dieser persönliche Offset auf. Angenommen, eine Person bewertet Filme generell eher schlechter, weil sie sich denkt, es geht immer etwas besser. Die Bewertungen dieser Person würde, im Vergleich zu einer Person, die bei Ihren Bewertungen auch mal eine volle Punktzahl gibt, mit einem euklidischen Ähnlichkeitsmaß als eher unähnlich angesehen werden.
Bei der Pearsonähnlichkeit werden Korrelationen bei Vielfachen erkannt. Bsp:
  > Person A bewertet drei Filme mit (1, 1.5, 1), Person B bewertet dieselben drei Filme mit (3, 4.5, 3)<br>
  > die Pearsonähnlichkeit würde Person A und Person B als maximal ähnlich ansehen, da alle Bewertungen von B ein Vielfaches (3-faches) der entsprechenden Bewertungen von A sind.<br>
  
Sieht man bei der Pearsonähnlichkeit die numerischen Werte, der zu vergleichenden Vektoren, als x- und y-Koordinaten eines Koordinatensystems und betrachtet die Regressionsgerade zu allen entstehenden Punkten, gilt:
 - Liegen alle Punkte auf der Geraden, ist die Korrelation entweder maximal +1 (positive Steigung der Gerade) oder -1 (negative Steigung der Geraden).
     -  Ist der **Korrelationskoeffizient +1**, liegt eine **vollkommene Korrelation** vor. Auf das Beispiel von Bewertungen übertragen würde das bedeuten, dass Kunde A im Verhältnis gleich wie Kunde B bewertet.
     -  Ist der **Korrelationskoeffizient -1**, liegt eine **negative Korrelation** vor. Bei Bewertungen würde das bedeuten, umso besser Kunde A bewertet hat, desto schlechter hat Kunde B bewertet.
 - Umso stärker die Streuung der Punkte um die Regressionsgerade ist, desto näher liegt der Korrelationskoeffizient bei 0. Ist der **Koeffizient 0**, wären die Daten der Kunden **nicht korreliert** und man könnte keine Vorhersage der Bewertungen von Kunde B auf Basis der Bewertungen von Kunde A machen.

#### Aufgabe 3
Da bei unserem Anwendungsfall Bewertungen vorliegen, bei denen nicht bekannt ist, wie der persönliche Offset des Users ist, eignet sich die Pearsonähnlichkeit hier am besten. So können auch negative Korrelationen erkannt werden. Wenn bspw. zwei Nutzer jeden Film gegensätzlich bewerten, kann man davon ausgehen, dass die beiden Nutzer einen grundsätzlich unterschiedlichen Filmgeschmack - ggf. auch unterschiedlichen Humor - haben. 

__Aufgabe:__
0. Schreiben Sie eine Funktion _topMatches(prefs,person,similarity)_, welche für eine beliebige in _critics_ enthaltene Person die Ähnlichkeitswerte zu allen anderen Personen berechnet und in einer geordneten Liste zurück gibt. Der Funktion soll als Übergabeparameter auch die anzuwendende Ähnlichkeitsfunktion (_sim_euclid_ oder _sim_pearson_) übergeben werden können. Berechnen Sie mit dieser Funktion für jede Person die _top matches_, zunächst unter Verwendung der euklidischen- dann unter Verwendung der Pearson-Ähnlichkeit.
1. Geben Sie mit der implementierten Funktion die _top matches_ der Person Toby aus.
2. Vergleichen Sie die beiden Ähnlichkeitsmaße. Welches Ähnlichkeitsmaß erscheint Ihnen für diesen Anwendungsfall sinnvoller und warum?

#### Aufgabe 1
Die Funktion topMatches reduziert die übergebene Tabelle (prefs) um die Zeile (matchItem), zu der die Ähnlichkeiten aller anderen Zeilen berechnet werden soll. Mit dem Übergabeparameter *similarity* kann man festlegen, mit welchem Ähnlichkeitsmaß die Berechnung durchgeführt werden soll. Am Ende wird eine sortierte Liste mit allen Items und zugehörigem Ähnlichkeitswert zurückgegeben.

In [10]:
def topMatches(prefs, matchItem, similarity):
    prefsReduced = {key: value for key, value in prefs.items()
                    if key is not matchItem}
    similarities = {}
    for key, value in prefsReduced.items():
        if similarity == '_simeuclid':
            similarities.update({key:(sim_euclid(prefs, matchItem, key))})
        elif similarity == '_simpearson':
            similarities.update({key:(sim_pearson(prefs, matchItem, key))})
        elif similarity == '_simrussel':
            similarities.update({key:(sim_RusselRao(prefs, matchItem, key))})
        else:
            return {}
        
    list = sorted(similarities.items(), key=lambda t: t[1], reverse=True)

    return list

In [11]:
for key in critics.keys():
    ad = 117
    d = (ad - key.__len__()-2)/2
    de = (ad - 28)/2
    dp = (ad - 26)/2
    print '-'*d,key,'-'*d
    print '  ','-'*de,'EUCLIDEAN SIMILARITIES','-'*de,'  \n', topMatches(critics,key,'_simeuclid'),'\b'
    print '  ','-'*dp,'PEARSON SIMILARITIES','-'*dp,'  \n', topMatches(critics,key,'_simpearson'),'\n'
print '-'*ad

--------------------------------------------------- Jack Matthews ---------------------------------------------------
   -------------------------------------------- EUCLIDEAN SIMILARITIES --------------------------------------------   
[('Gene Seymour', 0.9090909090909091), ('Lisa Rose', 0.7208254886814803), ('Mick LaSalle', 0.6666666666666666), ('Claudia Puig', 0.6534537935444722), ('Michael Phillips', 0.6534537935444722), ('Toby', 0.5227744249483389)] 
   --------------------------------------------- PEARSON SIMILARITIES ---------------------------------------------   
[('Gene Seymour', 0.9637956818756331), ('Lisa Rose', 0.7470178808339961), ('Toby', 0.6628489803598703), ('Mick LaSalle', 0.21128856368212914), ('Michael Phillips', 0.13483997249264842), ('Claudia Puig', 0.02857142857142857)] 

--------------------------------------------------- Mick LaSalle ---------------------------------------------------
   -------------------------------------------- EUCLIDEAN SIMILARITIES -----

#### Aufgabe 2

In [12]:
print '-'*36,'top matches for Toby (euclidean similarity)','-'*36,'\n', topMatches(critics, 'Toby','_simeuclid'),'\n'
print '-'*37,'top matches for Toby (pearson similarity)','-'*37,'\n', topMatches(critics, 'Toby','_simpearson'),'\n'
print '-'*117

------------------------------------ top matches for Toby (euclidean similarity) ------------------------------------ 
[('Mick LaSalle', 0.6666666666666666), ('Claudia Puig', 0.6246387977050463), ('Lisa Rose', 0.615911621788925), ('Michael Phillips', 0.5584815598877471), ('Jack Matthews', 0.5227744249483389), ('Gene Seymour', 0.5108747069239427)] 

------------------------------------- top matches for Toby (pearson similarity) ------------------------------------- 
[('Lisa Rose', 0.9912407071619302), ('Mick LaSalle', 0.9244734516419051), ('Claudia Puig', 0.8934051474415644), ('Jack Matthews', 0.6628489803598703), ('Gene Seymour', 0.3812464258315117), ('Michael Phillips', -1.0)] 

---------------------------------------------------------------------------------------------------------------------


#### Aufgabe 3
*siehe Antwort der vorangegangenen Aufgabe 3*. Unsere Bewertung der Ähnlichkeitsmaße hat sich nicht veändert.

Betrachtet man die Ähnlichkeitswerte und vergleicht sie mit den Bewertungen, die jeweilige User abgegeben haben, lassen sich folgende Punkte feststellen:

> Die User Toby und Michael Phillips haben nur zwei Filme beide bewertet. Dabei hat der eine den einen besser, der andere den anderen besser bewertet. Während die euklidiche Ähnlichkeit bei ca. 0.56 liegt, liegt der Wert der Pearsonkorrelation bei -1, was bedeutet, dass die beiden als negativ korreliert angesehen werden. Eine negative Korrelation zweier User würde bedeuten, dass diese gegenläufige Interessen haben. Jedoch kann das bei nur zwei bewerteten Filmen nicht zuverlässig gesagt werden. Man kann also davon ausgehen, dass eine zuverlässige Pearsonkorrelation nur mit einer entsprechenden Menge Daten zuverlässig ist.

> Auch für die User Michael Phillips und Mick LaSalle ergibt sich bei der Pearsonähnlichkeit ein negativer Wert (-0.26). Sie haben in ihren Bewertungen eine Schnittmenge von 4 Filmen. Dabei haben beide jeweils 2 Filme höher als der andere bewertet. Trotzdem werden die beiden nicht als maximal negativ korreliert - wie Michael Phillips und Toby - angesehen. Das bestätigt unsere Theorie, dass mit einer größeren Menge Daten die Pearsonkorrelation zurverlässiger ausfällt.

> Bewerten zwei User alle Filme annähernd gleich, wie die User Jack Matthews und Gene Seymour, wird die Pearsonkorrelation die User auch als stark korreliert ansehen. Aber auch wenn zwei User Filme im Verhältnis gleich bewertet haben (Beispiel in Lösung Afg. 2, vorangegenagener Aufgabenblock) wird das Ergebnis eine hohe Pearsonkorrelation sein.

### Berechnung von Empfehlungen mit User basiertem Collaborative Filtering
Für die Produkte, die von einer Person noch nicht gekauft wurden, sollen Empfehlungen berechnet werden. Die Empfehlungen können ebenfalls Werte zwischen 1 (wird nicht empfohlen) und 5 (wird stark empfohlen) annehmen. Für die Berechnung der Empfehlung werden die Bewertungen des jeweiligen Produkts durch die anderen Personen herangezogen. Dabei werden die Bewertungen der ähnlichen Personen (d.h. hoher Pearson-Korrelationswert) stärker mit einbezogen als die Bewertungen durch Personen mit einem niedrigen Korrelationswert.

__Beispiel:__
Toby hat die Filme _The Night Listener_, _Lady in the Water_ und _Just My Luck_ noch nicht gekauft. Für diese Filme soll für Toby eine Empfehlung berechnet werden.
In der unten aufgeführten Tabelle enthält die zweite Spalte die _Pearson-Ähnlichkeitswerte_ zwischen Toby und den anderen Personen. Die Spalten 3, 5 und 7 enthalten die Bewertungen der Filme _The Night Listener_, _Lady in the Water_ und _Just My Luck_ durch die anderen Personen. Die Spalten 4, 6 und 8 enthalten die jeweilige Filmbewertung gewichtet (mulipliziert) mit den Ähnlichkeitswerten der jeweiligen Person. Es fällt auf, dass in der Tabelle _Michael_ nicht enthalten ist. Das liegt daran, dass _Michael_ und _Toby_ einen negativen Ähnlichkeitswert aufweisen, d.h. deren Interessen sind gegenläufig. Personen mit negativem Ähnlichkeitswert sollten für Empfehlungen nicht berücksichtigt werden.
Die Zeile _Sum_ enthält die Summe aller gewichteten Bewertungen. Aus diesem Wert allein kann die Empfehlung noch nicht abgeleitet werden, da Filme die nur von wenigen Personen bewertet wurden, eine relativ kleine Summe ergeben. Deshalb sollte _Sum_ noch durch die Anzahl der Bewertungen für diesen Film geteilt werden. Oder besser: Nicht durch die Summe der Bewertungen, sondern durch die Summe der relevanten Ähnlichkeitswerte (_KSum_). Der resultierende Empfehlungswert ist in der letzten Zeile eingetragen.


![Abbildung Calculate Recommendation](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining/Bilder/recommenderFilmCalculation.PNG)


__Aufgabe:__
Schreiben Sie eine Funktion _getRecommendations(prefs,person,similarity)_, mit der die Empfehlungswerte berechnet werden können und bestimmen Sie die Empfehlungswerte für Toby. Der Funktion wird  

* das Dictionary _critics_ mit den Filmbewertungen, 
* der Name der Person, für welche Empfehlungen berechnet werden sollen
* die Methode für die Berechnung der Ähnlichkeit *sim_euclid* oder *sim_pearson*

übergeben. Die Methode soll eine geordnete Liste zurück geben. Jedes Listenelement enthält an erster Stelle den berechneten Empfehlungswert und an zweiter Stelle den Namen des Films. Die Liste soll nach Empfehlungswerten absteigend geordnet sein.

Testen Sie diese Funktion indem Sie die Empfehlungen für _Toby_ berechnen und mit den Werten in der oben aufgeführten Tabelle vergleichen.

#### Anmerkung

Wie oben beschrieben, werden alle drei Kriterien an die Funktion übergeben und als geordnete Liste zurückgegeben. Unsere Liste wird in absteigender Reihenfolge(Empfehlungswert) für noch nicht gesehene Filme für die gegebene Person ausgegeben. Ebenso wird anderst wie oben beschrieben der Name an erster Stelle und der berechnete Empfehlungswert an zweiter Stelle ausgegeben. Dies dient der besseren Lesbarkeit.

##### Codebeschreibung

In der gesamten Funktion wird darauf geachtet, dass keine Ähnlichkeiten einer Person zu sich selbst berechnet werden. 
> if candidate == person:
            continue
            
Für die übergebene Person wird zu jeder anderen Person die Ähnlichkeit berechnet und in ein Dictionary (sim) geschrieben.
Über alle Personen, die keine negative Korrelationen zum dem User haben, dem die Empfehlung gemacht werden soll, wird  die Korrelationssumme (kSums) gebildet und das Produkt aus Korrelationskoeffizient und Filmbewertung aller Filme, die der User noch nicht gesehen, bzw. bewertet hat, aufsummiert (items in unknownMedia).
Anschließend wird die Summe zu jedem Film durch die Summer der Korrelationskoeffizienten geteilt und das Ergebnis ist der finale Empfehlungswert zu jedem Film für den User. Zurückgegeben wir eine absteigend sortierte Liste.

In [13]:
def getRecommendations(prefs, person, similarity):

    #01 compute correlations
    sim = {}
    for candidate in prefs:
        if candidate == person:
            continue
        if similarity == '_simeuclid':
            sim[candidate] = sim_euclid(prefs, person, candidate)
        elif similarity == '_simpearson':
            sim[candidate] = sim_pearson(prefs, person, candidate)
        elif similarity == '_simrussel':
            sim[candidate] = sim_RusselRao(prefs,person,candidate)
        else:
            print ("no valid similarity")

    kSums = {}
    unknownMedia = {}
    for candidate in prefs:
        #02 don't compare a person with itself
        if candidate == person:
            continue
        #03 if the correlation is negative, the persons are too different
        if sim[candidate] < 0:
            continue
        #04 for every media the candidate already knows
        for media in prefs[candidate]:
            #05 for every media the candidate did not know
            if media not in prefs[person] or prefs[person][media] == 0:
                #06 check if the not yet seen media is already in the unknownMedia list
                #   if not, add it to unknownMedia and kSums with value 0
                if media not in unknownMedia:
                    unknownMedia[media] = 0
                    kSums[media] = 0
                #07 add the correlation of the candidate to the kSum of the current media
                kSums[media] += sim[candidate]
                #08 add the recommendation
                unknownMedia[media] += sim[candidate] * prefs[candidate][media]

    #09 divide the sum of the media by the kSum of the media
    for media in unknownMedia:
        unknownMedia[media] = unknownMedia[media]/kSums[media]


    #10 sort dictionary into list by descending keys (recommendation score)
    list = sorted(unknownMedia.items(), key=lambda t: t[1], reverse=True)

    return list

#### Berechnung der Empfehlungen für Toby mithilfe der oben implementierten Funktion *getRecommendations* auf Basis euklidischer Ähnlichkeitswerte

In [14]:
user = 'Toby'

print '-'*40, 'User based collaborative filtering', '-'*41
print 'Recommendations for %s (euclidean similarity): \n' % (user), getRecommendations(critics, user, '_simeuclid')
print '-'*117

---------------------------------------- User based collaborative filtering -----------------------------------------
Recommendations for Toby (euclidean similarity): 
[('The Night Listener', 3.4273481378103883), ('Lady in the Water', 2.7957370311640055), ('Just My Luck', 2.407392750287351)]
---------------------------------------------------------------------------------------------------------------------


#### Berechnung der Empfehlungen für Toby mithilfe der oben implementierten Funktion *getRecommendations* auf Basis  der Pearsonähnlichkeit

In [15]:
user = 'Toby'

print '-'*40, 'User based collaborative filtering', '-'*41
print 'Recommendations for %s (pearson similarity): \n' % (user), getRecommendations(critics, user, '_simpearson')
print '-'*117

---------------------------------------- User based collaborative filtering -----------------------------------------
Recommendations for Toby (pearson similarity): 
[('The Night Listener', 3.347789526713101), ('Lady in the Water', 2.8325499182641622), ('Just My Luck', 2.5309807037655645)]
---------------------------------------------------------------------------------------------------------------------


### Berechnung von Empfehlungen mit Item basiertem Collaborative Filtering
In den vorigen Aufgaben wurden Ähnlichkeiten zwischen Personen bestimmt und für Produktempfehlungen benutzt (User basiertes Collaborative Filtering). Jetzt soll die Ähnlichkeit zwischen Produkten berechnet werden und auf der Basis dieser Produktähnlichkeit Empfehlungen berechnet werden (Item basiertes Collaborative Filtering).

Dabei sollen die bereits implementierten Ähnlichkeitsfunktion *sim_euclid* und *sim_pearson* sowie die Ähnlichkeeits-Sortierfunktion *topMatches* unverändert eingesetzt werden.

__Aufgabe:__

1. Implementieren Sie eine Funktion, welche das Bewertungsdictionary *critics* derart transformiert, dass die Funktionen *sim_euclid*, *sim_pearson* und *topMatches* für das Item-basierte CF unverändert eingesetzt werden können. Die transformierte Matrix soll unter dem Namen *transCritics* abgespeichert werden.
2. Schreiben Sie eine Funktion *calculateSimilarItems*, die aus der transformierten Matrix *transCritics* ein Dictionary berechnet, welches die Ähnlichkeit zwischen allen Filmen beschreibt. Die Keys des Dictionary sind die Filmnamen. Die Values sind geordnete Listen, welche die Funktion *topMatches* zurückgibt, wenn sie für die Filme (nicht für die User) aufgerufen wird. Dieses Dictionary wird an das aufrufende Programm zurück geben. 
3. Schreiben Sie eine Funktion *getRecommendedItems*, welche basierend auf dem im unten aufgeführten Beispiel dargestellten Verfahren unter Vorgabe der Bewertungsmatrix und der zu verwendenden Ähnlichkeitsfunktion Produktempfehlungen berechnet.
4. Testen Sie die Funktion indem Sie die Empfehlungen für Toby berechnen und mit den Werten in der unten aufgeführten Tabelle vergleichen

__Erläuterndes Beispiel:__

_Toby_ hat die Filme *The Night Listener*, *Lady in the Water* und *Just My Luck* noch nicht gekauft. Für diese Filme soll für *Toby* eine Empfehlung berechnet werden. Gekauft und bewertet hat *Toby* die Filme *Snakes on a plane*, *Superman Returns* und *You and me and Dupree*. Diese bereits vorhandenen Filme bilden die erste Spalte der unten dargestellten Matrix. In der zweiten Spalte befinden sich _Toby's_ Bewertungen dieser Filme. Die Spalten 3,5 und 7 enthalten die Ähnlichkeitswerte (mit *calculateSimilarItems* unter Verwendung des normierten euklidischen Ähnlichkeitsmaßes berechnet) zwischen den drei von *Toby* noch nicht gekauften Filmen und den drei von _Toby_ bewerteten Filmen. Diese Ähnlichkeitswerte werden jeweils mit _Toby's_ Bewertungen multipliziert. Das Resultat dieser Multiplikation befindet sich in den Spalten 4,6 und 8. Der finale Empfehlungswert für die von _Toby_ noch nicht gekauften Filme wird berechnet in dem in den Spalten 4,6 und 8 zunächst die Summe über die Werte dieser Spalte in den drei oberen Zeilen berechnet wird und durch die Summe über die Werte der Spalten 3,5 und 7 geteilt wird. Im Fall, dass die *Pearson-Korrelation* zwischen den Filmen als Ähnlichkeitswert herangezogen wird, können negative Ähnlichkeitswerte auftreten. Dann soll in die Berechnung eines Empfehlungswert für Film A nur dann die Bewertung von Film B einfließen, wenn der Korrelationswert zwischen beiden $>0$ ist.  

![Abbildung Calculate Itembased Recommendation](https://www.hdm-stuttgart.de/~maucher/ipnotebooks/DataMining/Bilder/recommenderFilmItemBased.PNG)

#### Anmerkung

Auch hier wird an erster Stelle der Liste der Name ausgegeben und an zweiter Stelle der Empfehlungswert. Dies ist dient der besseren Lesbarkeit.

Der Nachfolgende Code ist in seine obigen Teilaufgaben eingeteilt.

#### Aufgabe 1

##### Codebeschreibung

Ziel: Ein neues verschachteltes Dictionary (movies), in die Keys die Filmtitel sind und die Werte Dictionariies mit den User-Bewertungspaaren zum jeweilgen Film.

1. Iteration über alle Key-Value-Paare des Dictionary critics mit den Bewertungen jedes Users zu den Filmen.

2. Iteration über alle Key-Value-Paare im verschachtelten Dictionary, das die Bewertungen zu jedem Film enthält.

3. Wenn der aktuelle Film noch nicht im neuen Dictionary als Key existiert, lege ihn neu an.

4. Schreibe in das innere Dictionary des jeweiligen Films den Namen des Users als Key und seine Bewertung als Wert.

In [16]:
def transCritics(critics):
    movies = {}

    #1
    for critics, details in critics.items():
        #2
        for title, rating in details.items():
            #3
            if title not in movies:
                movies[title] = {}
            #4
            movies[title][critics] = rating

    return movies

print '-'*46,'transformation Critics','-'*47
print transCritics(critics)
print '-'*117

---------------------------------------------- transformation Critics -----------------------------------------------
{'Lady in the Water': {'Lisa Rose': 2.5, 'Jack Matthews': 3.0, 'Michael Phillips': 2.5, 'Gene Seymour': 3.0, 'Mick LaSalle': 3.0}, 'Snakes on a Plane': {'Jack Matthews': 4.0, 'Mick LaSalle': 4.0, 'Claudia Puig': 3.5, 'Lisa Rose': 3.5, 'Toby': 4.5, 'Gene Seymour': 3.5, 'Michael Phillips': 3.0}, 'Just My Luck': {'Claudia Puig': 3.0, 'Lisa Rose': 3.0, 'Gene Seymour': 1.5, 'Mick LaSalle': 2.0}, 'Superman Returns': {'Jack Matthews': 5.0, 'Mick LaSalle': 3.0, 'Claudia Puig': 4.0, 'Lisa Rose': 3.5, 'Toby': 4.0, 'Gene Seymour': 5.0, 'Michael Phillips': 3.5}, 'The Night Listener': {'Jack Matthews': 3.0, 'Mick LaSalle': 3.0, 'Claudia Puig': 4.5, 'Lisa Rose': 3.0, 'Gene Seymour': 3.0, 'Michael Phillips': 4.0}, 'You, Me and Dupree': {'Jack Matthews': 3.5, 'Mick LaSalle': 2.0, 'Claudia Puig': 2.5, 'Lisa Rose': 2.5, 'Toby': 1.0, 'Gene Seymour': 3.5}}
---------------------------------

#### Aufgabe 2

##### Codebeschreibung

In transformierter Matrix wird über jeden Filmtitel iteriert und mit der Funktion *topMatches* alle Ähnlichkeitswerte berechnet und in einem Dictionary (simItems) gespeichert.

In [17]:
def  calculateSimilarItems(prefs, similarity):
    simItems = {}

    for title in prefs.keys():
        simItems[title] = topMatches(prefs,title,similarity)

    return simItems

print '-'*41,'Similarities of calculated items','-'*42
print calculateSimilarItems(transCritics(critics), '_simpearson')
print '-'*117

----------------------------------------- Similarities of calculated items ------------------------------------------
{'Lady in the Water': [('Snakes on a Plane', 0.7637626158259734), ('Superman Returns', 0.48795003647426655), ('You, Me and Dupree', 0.3333333333333333), ('The Night Listener', -0.6123724356957947), ('Just My Luck', -0.9449111825230683)], 'Snakes on a Plane': [('Lady in the Water', 0.7637626158259734), ('Superman Returns', 0.11180339887498948), ('Just My Luck', -0.3333333333333333), ('The Night Listener', -0.5663521139548542), ('You, Me and Dupree', -0.6454972243679029)], 'Just My Luck': [('The Night Listener', 0.5555555555555556), ('Snakes on a Plane', -0.3333333333333333), ('Superman Returns', -0.422890031611031), ('You, Me and Dupree', -0.48566186425718266), ('Lady in the Water', -0.9449111825230683)], 'Superman Returns': [('You, Me and Dupree', 0.657951694959769), ('Lady in the Water', 0.48795003647426655), ('Snakes on a Plane', 0.11180339887498948), ('The Night List

#### Aufgabe 3

##### Codebeschreibung

Berechnung Filmempfehlung
Es wird eine Liste erstellt, die alle Filme enthält, die der User, dem die Empfehlung gemacht werden soll, noch nicht gesehen/bewertet hat (notboughtMovies). Anschließend wird für jeden Film in dieser Liste der Empfehlungswert wie folgt berechnet:
> Für jeden vom User gekauften Film wird die Ähnlichkeit zum nicht gekauften Film mit dem angegebenen Ähnlichkeitsmaß berechnet. Diese Ähnlichkeit wird anschließend mit der Bewertung des Films durch den User multipliziert und zur Empfehlungswert addiert. Gleichzeitig werden die Ähnlichkeitswerte des nicht gekauften Films zu allen gekauften Filmen aufsummiert. Am Ende wird der Empfehlungswert durch eine Divison durch die Summe der Ähnlichkeitswerte normiert.

Die normierten Empfehlungswerte werden in ein Dictionary zu den zugehörigen Filmtiteln geschrieben. Zurückgegeben wird eine absteigend sortierte Liste.

In [18]:
def getRecommendedItems(prefs,person,similarity):
    simItem = calculateSimilarItems(prefs,similarity)
    result = []

    notboughtMovies = []
    for movie in simItem.keys():
        if movie not in critics[person]:
            notboughtMovies.append(movie)

    recommandations = {}
    for notboughtMovie in notboughtMovies:
        sumRecommendation = 0
        sumSimilarities = 0
        for purchasedMovie, rating in critics[person].items():
            if similarity == '_simeuclid':
                simi = sim_euclid(prefs, purchasedMovie, notboughtMovie)
                sumRecommendation += simi * rating
                sumSimilarities += simi
            elif similarity == '_simpearson':
                simi = sim_pearson(prefs, purchasedMovie, notboughtMovie)
                sumRecommendation += simi * rating
                sumSimilarities += simi
        #3
        recommandations[notboughtMovie] = (sumRecommendation / 
                                           sumSimilarities).round(2) if sumSimilarities != 0 else 0

        list = sorted(recommandations.items(), key=lambda t: t[1], reverse=True)

    return list

#### Aufgabe 4

##### Codebeschreibung
Transformieren der Ausgangsmatrix. <br>
Item-Based Empfehlung für Toby.

In [19]:
tc = transCritics(critics)
person = 'Toby'

print '-'*40, 'Item based collaborative filtering', '-'*41
print 'Recommendations for %s (euclidean similarity): \n' % (person), getRecommendedItems(tc, person, '_simeuclid')
print 'Recommendations for %s (pearson similarity): \n' % (person), getRecommendedItems(tc, person, '_simpearson')
print '-'*117

---------------------------------------- Item based collaborative filtering -----------------------------------------
Recommendations for Toby (euclidean similarity): 
[('The Night Listener', 3.2), ('Lady in the Water', 3.08), ('Just My Luck', 3.04)]
Recommendations for Toby (pearson similarity): 
[('Lady in the Water', 3.61), ('The Night Listener', 3.53), ('Just My Luck', 2.96)]
---------------------------------------------------------------------------------------------------------------------


## last.fm Musikempfehlungen
Kopieren Sie die Datei _pylast.py_ vom _Resources_-Ordner im _DataMining_-Ordner des Skripteservers in das Verzeichnis dieses _IPython Notebooks_. In dieser Datei sind alle Zugriffsfunktionen auf _last.fm_ Dienste implementiert. Die notwendigen Anmelde- und Authentifizierungsdaten für den User _pythonlab_ sind ebenfalls schon in diesem Modul eingetragen.

__Aufgabe:__

1. Stellen Sie durch Aufruf der Funktion *network=pylast.get_lastfm_network()* eine Verbindung zu *last.fm* her. Beim Aufruf der Funktion wird die Anmeldung und Authentifizierung durchgeführt. Die Funktion gibt ein Objekt der Klasse *Network* zurück. Über dieses Objekt können Methoden, wie

    * *get_artist("kuenstlerName")* (liefert Objekt der Klasse _Artist_)
    * *get_album("albumName")* (liefert Objekt der Klasse _Album_)
    * *get_track("songName")* (liefert Objekt der Klasse _Track_)
    * *get_user("userName"):* (liefert Objekt der Klass_Tag_)
    * usw.
    
      aufgerufen werden. Die Menge aller verfügbaren Klassen und deren Attribute und Methoden können dem Modul _pylast.py_ entnommen werden.

1. Rufen Sie über das oben instanziierte _Network_-Objekt die Methode *get_artist("BandIhrerWahl")* auf.
2. Rufen Sie über das oben instanziierte _Artist_-Objekt die Methode *topfans=get_top_fans(10)* auf. Die Methode gibt eine Liste von _User_-Objekt/Gewichtung-Paaren zurück. Die Gewichtungen von Objekten werden in diesem Versuch nicht benötigt. Legen Sie deshalb mit _group=[a.item for a in topfan]_ eine Liste an, die nur noch die User Objekte enthält. **Wichtige Anmerkung:** Seit August 2015 gibt es Probleme mit der lastFM API Methode *get_top_fans()* (siehe auch: [pylast issues](https://github.com/pylast/pylast/issues/155s)). Falls am Versuchstermin der Fehler noch nicht behoben ist, können Sie den unten stehenden Code benutzen. Darin wird versucht auf die API-Methode zuzugreifen. Falls das nicht möglich ist, wird eine vordefinierte Liste von Usern angewandt. Diese Liste repräsentiert die *Top Fans* der Band *Slipknot* im Frühjahr 2015. 
3. Implementieren Sie eine Funktion *createLastfmUserDict()*. Dieser Funktion soll, die oben angelegte Liste von _User_-Objekten _group_ übergeben werden. Für jeden User in *group* sollen die 20 beliebtesten Bands mit der Methode *topartists=get_top_artists()[0:20]* bestimmt werden. Die Methode gibt eine Liste von *Artist*-Objekt/Gewichtung-Paaren zurück. Die Gewichtungen von Objekten werden in diesem Versuch nicht benötigt. Auf das *i.te* *Artist*-Objekt selbst kann mit *topartists[i].item* zugegriffen werden. Die Menge aller Bands, die auf diese Weise gesammelt werden, wird im folgenden mit _AllBands_ bezeichnet. D.h. in *AllBands* befinden sich alle Bands, die für mindestens einen User in *group* zu den Top-20 gehören. Nun soll ein verschachteltes Dictionary mit Namen *userDict* wie folgt angelegt werden:

    * Die Keys sind die Namen der _User_-Objekte in _group_. Auf den Namen eines Objekts kann mit *get_name()* zugegriffen werden.
    * Die Values sind selbst wieder Dictionaries, deren Keys die Namen der Bands in *AllBands* sind. Achten Sie auch hier darauf, dass Sie nicht das *Artist*-Objekt selbst, sondern dessen Namen als Key verwenden. 
    * Für den User *a* und die Band *b* ist der Value *userDict[a][b]= 1*, falls *b* zu den Top-20 des Users *a* gehört. Andernfalls ist *userDict[a][b]= 0*. 
    
    Das derart angelegte Dictionary soll von der Funktion zurückgegeben werden. 
4. Wählen Sie jetzt einen beliebigen User aus *group*. Bestimmen Sie zu diesem User die ähnlichsten User in *group* durch Anwendung der im ersten Teilversuch implementierten Funktion *topMatches()*. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik angewandt werden.
5. Bestimmen Sie dann für den gewählten User Band-Empfehlungen durch Anwendung der im ersten Teilversuch implementierten Funktion *getRecommendations()*. Der Funktion wird das angelegte *userDict* und der Name des gewählten Users übergeben. Als Ähnlichkeitsmaß soll die euklidische Metrik, danach die Russel_Rao Metrik, angewandt werden.     
6. Diskutieren Sie das Ergebnis

In [20]:
import pylast
nw=pylast.get_lastfm_network(api_key = "993a5bd9d79a98a53677570368d55acd",
                             api_secret = "9b8de0b57903ac007cdd8ec9003b341e",username = "pythonlab")
band='Slipknot'
art1 = nw.get_artist(band)
print "Most similar (as calculated by lastFM) for artist: ",band
for it in art1.get_similar(5):
    print "%3.3f \t %s"%(it.match, it.item)
try:
    topfan = art1.get_top_fans()
    group = [a.item for a in topfan]
except:
    print "\nlastFM API Error for method get_top_fans\nApply predefined group of users"
    usernames=['BrunoJoS','DPREBOYE','MPistol40','NemoNightfall','SkyRif','Wags1382',
               'Znapsen','cortapsyco','emill_67','sattuviitana']
    group=[]
    for u in usernames:
        u1 = nw.get_user(u)
        group.append(u1)

Most similar (as calculated by lastFM) for artist:  Slipknot
1.000 	 Stone Sour
0.697 	 Korn
0.498 	 Mudvayne
0.449 	 System of a Down
0.415 	 Limp Bizkit

lastFM API Error for method get_top_fans
Apply predefined group of users


#### Anmerkung

Wie oben beschrieben, funktioniert der Aufruf der lastFM Methode .get_top_fans nicht mehr, so das der unten stehende Code nur als Pseudocode dient. Ebenso wird daher die vordefinierte Liste der User, für das weitere vorgehen genutzt.

#### Aufgabe 1 - 3

In [21]:
import pylast as pl

network = pl.get_lastfm_network()

artist = network.get_artist('Drake')

# topfans = artist.get_top_fans(10)

# print topfans

#### Aufgabe 4

##### Codebeschreibung

1. Für jeden User werden die ersten X favorisierten Artists ermittelt (X wird durch Übergabeparameter *maxArtists* festgelegt).

2. Jeder lesbare Artist (nur lateinische Schriftzeichen) wird zum einen in die topBandList des Users geschrieben und    zum anderen einem Set hinzugefügt (allBands), das alle Artists enthält, die einer der User unter seinen Favoriten hatte.

3. Anschließend wird nochmal über jeden User iteriert und für jeden Artist des allBands-Set geprüft, ob er vom User favorisiert ist. Diese Information wird in einer User-Artist-Matrix gespeichert, in der nur 0 (nicht vom User favorisiert) oder 1 (vom User favorisiert) vorkommen.
Diese Matrix wird von der Funktion zum Schluss zurückgegeben.

In [22]:
def createLastfmUserDict(group, maxArtist):
    allBands = set()
    topUserBands = {}
    userDict = {}

    #1
    for user in group:
        topartist = user.get_top_artists()[0:maxArtist]
        topBandList = []
        for i in range(0, maxArtist):
            #2
            try:
                allBands.add(str(topartist[i].item.name))
                topBandList.append(str(topartist[i].item.name))
            except:
                print "not able to write", topartist[i].item.name, "band into set"
        topUserBands[user] = topBandList
    #3
    for user in group:
        bandDict = {}
        for band in allBands:
            if band in topUserBands[user]:
                bandDict[band] = 1
            else:
                bandDict[band] = 0
        userString = str(user)
        userDict[userString] = bandDict

    return userDict

#1
maxArtist = 20
userDict = createLastfmUserDict(group, maxArtist)

not able to write 植松伸夫 band into set


#### Aufgabe 5

Wählen eines Users und Anzeigen der Ähnlichsten User.

In [23]:
user = 'SkyRif'

d = (117 - user.__len__() -2)/2

print '-'*d,user,'-'*d,'\n'
print '-'*52,'TOP MATCHES','-'*51
print '-'*47,'Euclidean similarity','-'*47,'\n', topMatches(userDict, user,'_simeuclid')
print '-'*117

------------------------------------------------------ SkyRif ------------------------------------------------------ 

---------------------------------------------------- TOP MATCHES ---------------------------------------------------
----------------------------------------------- Euclidean similarity ----------------------------------------------- 
[('MPistol40', 0.9657045104091802), ('NemoNightfall', 0.9645434752296831), ('sattuviitana', 0.9645434752296831), ('Znapsen', 0.9634231907991175), ('BrunoJoS', 0.9634231907991175), ('emill_67', 0.9634231907991175), ('cortapsyco', 0.9618110670539636), ('DPREBOYE', 0.9612903225806452), ('Wags1382', 0.9602717282659217)]
---------------------------------------------------------------------------------------------------------------------


#### Aufgabe 6
Berechnung der Empfehlungen mithilfe der bereits implementierten Funktion *getRecommendations* auf Basis der euklidischen Ähnlichkeit und der Russel-Rao-Metrik.

In [24]:
print '-'*50,'RECOMMENDATIONS','-'*50
print '-'*47,'Euclidean similarity','-'*48,'\n', getRecommendations(userDict, user, '_simeuclid'), '\b'
print '-'*47,'Russel_Rao similarity','-'*47,'\n', getRecommendations(userDict, user, '_simrussel')
print '-'*117

-------------------------------------------------- RECOMMENDATIONS --------------------------------------------------
----------------------------------------------- Euclidean similarity ------------------------------------------------ 
[('Five Finger Death Punch', 0.44482466669470455), ('Rammstein', 0.33355388140531134), ('In Flames', 0.22267550886097784), ('Limp Bizkit', 0.22254627162954374), ('Three Days Grace', 0.22241233334735228), ('30 Seconds to Mars', 0.22241233334735228), ('Avenged Sevenfold', 0.22241233334735228), ('Pantera', 0.22241233334735228), ('Pendulum', 0.22236029527937015), ('Muse', 0.22228309611591818), ('Pink Floyd', 0.22228309611591818), ('Foo Fighters', 0.22228309611591818), ('Arctic Monkeys', 0.22203704611642833), ('Coheed and Cambria', 0.22173356361728241), ('Team Sleep', 0.11140472357158465), ('Soilwork', 0.11140472357158465), ('A Perfect Circle', 0.11140472357158465), ('Soulfly', 0.11140472357158465), ('36 Crazyfists', 0.11140472357158465), ('Dmitri Shostakovi

#### Aufgabe 7

Beide Metriken ergeben ähnliche Empfehlungen. Wenn gesichert ist, dass zwei User definitiv gleiche Bands hören, ist die Verwendung der Russel-Rao-Metrik möglich. Da die Russel-Rao-Metrik für die Berechnung der Ähnlichkeit nur Auftreten von gemeinsamen nicht-Null Werten zählt, kann es passieren, dass bei kleinen Datenmengen mit geringen Korrelationen viele User als maximal Unähnlich angesehen werden und somit bei der Berechnung der Empfehlungen alle Werte mit 0 gewichtet werden und kein aussagekräftiges Ergebnis geliefert werden kann *(siehe weiterer Test, unten folgend)*. Es ist allgemein auch fraglich, wie aussagekräftig das Ergebnis ist, da wir nur die Top-20 Artists von zehn User für die Berechnung der Empfehlung verwenden. Alle diese User haben als gemeinsamme Band nur Slipknot. Dem gewählten User wird die Band *'Five Finger Death Punch'* empfohlen. Das heißt, dass diese Band die Band ist, die von den meisten der anderen neun anderen User gehört wird, aber nicht von unserem gewählten User selbst. 

Bei einem Test, bei dem wir zwei zufällige lastFM-Usernames (*joi* & *Larvi*) aus dem Internet herausgesucht, der Usergruppe hinzugefügt und die Berechnungen für einen dieser User durchgeführt haben, konnten wir feststellen, dass diesem User Slipknot empfohlen wird, da zehn von elf andere User Slipknot unter ihren Top-20 Bands haben. Der Code ist in der folgenden Zelle zu finden.
Die Empfehlungen, die bei der Verwendung der Russel Rao Metrik herauskommen, sehen so aus, da alle User bis auf 'BrunoJoS' keine Ähnlichkeit zum gewählten User aufweisen (unter TOP MATCHES - Russel Rao Similarity zu sehen). Deshalb werden bei der Berechnung der Empfehlungen die Bands dieser User alle mit 0 gewichtet. Somit entsprichen die Empfehlungen für den gewählten User nur den 19 Bands, die BrunoJoS unter seinen favorisierten 20 Artists hat, aber nicht unser gewählte User. Die Schnittmenge, die zu einer Russel-Rao-Ähnlichkeit von > 0 geführt hat, entspricht somit genau einer Band, die BrunoJoS und unser gewählte User beide favorisieren. Man kann davon ausgehen, wenn mehr Top Artists betrachtet werden, führt das zu einer besseren Performance der Russel-Rao-Metrik.

In [25]:
usernames2 = ['BrunoJoS','DPREBOYE','MPistol40','NemoNightfall','SkyRif','Wags1382','Znapsen',
              'cortapsyco','emill_67','sattuviitana', 'joi','Larvi']
group2 = []
    
for u in usernames2:
    u1 = nw.get_user(u)
    group2.append(u1)
        
userDict2 = createLastfmUserDict(group2, maxArtist)

user2 = 'joi'

d = (118 - user.__len__())/2

print '\b'
print '-'*d,user2,'-'*d,'\n'
print '-'*52,'TOP MATCHES','-'*52
print '-'*47,'Euclidean similarity','-'*48,'\n', topMatches(userDict2, user2,'_simeuclid')
print '-'*47,'Russel Rao Similarity','-'*47,'\n', topMatches(userDict2, user2,'_simrussel'),'\n'
print '-'*50,'RECOMMENDATIONS','-'*50
print '-'*47,'Euclidean similarity','-'*48,'\n', getRecommendations(userDict2, user2, '_simeuclid'), '\b'
print '-'*47,'Russel_Rao similarity','-'*47,'\n', getRecommendations(userDict2, user2, '_simrussel')
print '-'*117

not able to write 植松伸夫 band into set
not able to write Frédéric Chopin band into set

-------------------------------------------------------- joi -------------------------------------------------------- 

---------------------------------------------------- TOP MATCHES ----------------------------------------------------
----------------------------------------------- Euclidean similarity ------------------------------------------------ 
[('BrunoJoS', 0.9683325955428261), ('cortapsyco', 0.9679211469253941), ('Wags1382', 0.9675154200971697), ('DPREBOYE', 0.9675154200971697), ('MPistol40', 0.9675154200971697), ('Larvi', 0.9675154200971697), ('Znapsen', 0.9675154200971697), ('NemoNightfall', 0.9675154200971697), ('emill_67', 0.9675154200971697), ('SkyRif', 0.9675154200971697), ('sattuviitana', 0.9675154200971697)]
----------------------------------------------- Russel Rao Similarity ----------------------------------------------- 
[('BrunoJoS', 0.005376344086021506), ('cortapsyco', 0.0)

Bei einem weiteren Test wollten wir herausfinden, wie sich die Empfehlungen verändern, wenn man mehr Artists zu jedem User auslesen lässt. Dafür haben wir den Wert maxArtists auf 50 gesetzt. Das ist wohlmöglich der maximale Wert, der über die API von lastfm einstellbar ist. Bei größteren Werten über 50+ bekommen wir "Index out of Range" Errors die nur so erklärbar sind. Durch erweitertes debuggen des Codes, konnten wir Fehler in der Programmierung ausschließen.

Es ist interessant zu sehen, dass dem User 'SkyRif' bei der Verwendung der Top 50 Artists der User 'sattuviitana' am ähnlichsten ist, während bei 20 Artists der ähnlichste User 'MPistol40' war. Das lässt sich so erklären, dass bei der Betrachtung der Top 50 Artists die User 'SkyRif' und 'sattuviitana' die meisten gemeinsammen Favoriten hatten, während bei einer Betrachtung der Top 20 'MPistol40' eine größere Schnittmenge mit 'SkyRif' hatte. Daraus kann man schließen, dass 'SkyRif' und 'MPistol40' im Groben den ähnlichsten Musikgeschmack haben, sobald man aber ins Detail geht, eher der Musikgeschmack 'sattuviitana's' ähnlich ist.<br>
Außerdem fällt auf, dass der höchste Empfehlungswert zwar gleich bleibt (ca. 0.45), jedoch eine andere Band (*Rammstein*) empfohlen wird, als mit 20 Artists (*Five Finger Death Punch*). Das ist darauf zurückzuführen, dass mehr User *Five Finger Death Punch* unter ihren Top 20 haben, unter den Top 50 der User jedoch *Rammstein* häufiger vertreten ist.

In [26]:
maxArtist2 = 50
userDict3 = createLastfmUserDict(group, maxArtist2)

d = (117 - user.__len__()-2)/2

print '-'*d,user,'-'*d,'\n'
print '-'*52,'TOP MATCHES','-'*52
print '-'*47,'Euclidean similarity','-'*48,'\n', topMatches(userDict3, user,'_simeuclid'),'\n'
print '-'*50,'RECOMMENDATIONS','-'*50
print '-'*47,'Euclidean similarity','-'*48,'\n', getRecommendations(userDict3, user, '_simeuclid'), '\b'
print '-'*47,'Russel_Rao similarity','-'*47,'\n', getRecommendations(userDict3, user, '_simrussel')
print '-'*117

not able to write Мусоргский, Модест Петрович band into set
not able to write Glenmark Eriksson Strömstedt band into set
not able to write Kapten Röd band into set
not able to write Niklas Strömstedt band into set
not able to write 植松伸夫 band into set
not able to write Beyoncé band into set
not able to write Mötley Crüe band into set
not able to write Motörhead band into set
------------------------------------------------------ SkyRif ------------------------------------------------------ 

---------------------------------------------------- TOP MATCHES ----------------------------------------------------
----------------------------------------------- Euclidean similarity ------------------------------------------------ 
[('sattuviitana', 0.9768146849684994), ('BrunoJoS', 0.9763413909393326), ('Znapsen', 0.9752747252747253), ('MPistol40', 0.9752747252747253), ('emill_67', 0.974978927438057), ('NemoNightfall', 0.9745421627462424), ('DPREBOYE', 0.973972083153378), ('cortapsyco', 0.9736

Interessant wäre es außerdem auch, wie sich die Empfehlungen verändern, wenn man eine größere Menge User (100+) zur Berechnung der Empfehlungen verwendet. Leider wäre es äußerst aufwendig eine Liste mit 100 oder mehr Usern manuell zu erstellen. Theoretisch müsste für jeden User eine spezifischere Empfehlung getätigt werden können, da mehr Referenzdaten zur Verfügung stehen.